In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
from datasets.fbp_dataset import FBPDataset

dataset = FBPDataset()

100%|██████████| 15594/15594 [02:19<00:00, 111.95it/s]


In [4]:
batch_size = 5
num_queries = 100
num_classes = len(dataset.label_unique)

batch = [dataset[i] for i in range(batch_size)]
targets = [data[2] for data in batch]
len_sequences = torch.Tensor([data[1] for data in batch])
docs = [data[0] for data in batch]

predictions = {
    'pred_logits': torch.rand([batch_size, num_queries, num_classes + 1]) * torch.Tensor([[1] * num_classes + [10]]),
    'pred_boxes': torch.rand([batch_size, num_queries, 2]) * torch.Tensor([[1, 0.1]])
}

In [5]:
from models.matcher import HungarianMatcher

matcher = HungarianMatcher()

In [8]:
from models.criterion import CriterionDETR

weight_dict = {'loss_ce': 1., 'loss_bbox': 1., 'loss_giou': 1.}
losses = ['labels', 'boxes', 'cardinality']

criterion = CriterionDETR(num_classes, matcher=matcher, weight_dict=weight_dict,
                            eos_coef=0.1, losses=losses)

In [9]:
criterion(predictions, targets)

{'loss_ce': tensor(2.6913),
 'loss_bbox': tensor(0.0577),
 'loss_giou': tensor(0.4735),
 'cardinality_error': tensor(5.2000)}

In [12]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
labels = dataset.label_unique.reshape(-1, 1)
encoder2 = encoder.fit(labels)

In [23]:
from datasets import build_datasets_evaluator

train, val, evaluator = build_datasets_evaluator(False, 0.3, 42)

100%|██████████| 15594/15594 [00:03<00:00, 5197.50it/s]
